In [1]:
! pip install -U mock

! pip uninstall pytube

In [2]:
import re
import mock
import os

from pytube.cipher import get_throttling_function_code

from pytube import YouTube

def patched_throttling_plan(js: str):
    """Patch throttling plan, from https://github.com/pytube/pytube/issues/1498"""
    raw_code = get_throttling_function_code(js)

    transform_start = r"try{"
    plan_regex = re.compile(transform_start)
    match = plan_regex.search(raw_code)

    #transform_plan_raw = find_object_from_startpoint(raw_code, match.span()[1] - 1)
    transform_plan_raw = js

    # Steps are either c[x](c[y]) or c[x](c[y],c[z])
    step_start = r"c\[(\d+)\]\(c\[(\d+)\](,c(\[(\d+)\]))?\)"
    step_regex = re.compile(step_start)
    matches = step_regex.findall(transform_plan_raw)
    transform_steps = []
    for match in matches:
        if match[4] != '':
            transform_steps.append((match[0],match[1],match[4]))
        else:
            transform_steps.append((match[0],match[1]))

    return transform_steps


with mock.patch('pytube.cipher.get_throttling_plan', patched_throttling_plan):

    url = 'https://www.youtube.com/watch?v=VIR46oH-ufk&ab_channel=Horses'

    video = YouTube(url)
    audio = video.streams.filter(only_audio=True, file_extension='mp4')[0]

    dir = video.title
    check_dir = os.path.isdir(dir)
    if not check_dir:
        os.mkdir(video.title)
    else:
        pass

    audio.download(filename=video.title + '/' + video.title + '.mp4')

bullshit is fixed


In [3]:
! pip install openai-whisper 

In [5]:
import whisper

model = whisper.load_model("tiny.en")
content = model.transcribe(video.title + '/' + video.title + '.mp4', fp16=False)

with open(video.title + '/' + video.title + '.txt' , 'w') as f:
    f.write(content["text"])
    f.close()

In [6]:
# Installs langchain libraries
! pip install -U langchain langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 6.0 MB/s eta 0:00:00m eta 0:00:010:0101
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.4 MB/s eta 0:00:0031m11.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.23.6
    Uninstalling openai-1.23.6:
      Successfully uninstalled openai-1.23.6
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.46
    Uninstalling langchain-core-0.1.46:
      Successfully uninstalled langchain-core-0.1.46
  Attempting uninstall: langchain_openai
    Found existing installation: langchain-openai 0.1.4
    Uninstalling langchain-openai-0.1.4:
      Successfully uninstalled langchain-openai-0.1.4
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.0.34
 

In [7]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI


from langchain.cache import SQLiteCache
from langchain.globals import set_llm_cache
set_llm_cache(SQLiteCache(database_path=".langchain.db"))


from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()




model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5)

prompt = PromptTemplate(
    # template="""Create 3 separate paragraphs based on this text, start the first paragraph with the words /
    # 'The idea of ​​this text...' and make it one sentence long, start second paragraph with 'Content of this text...' and provide shortened version of originat provided text, make it at least 8 sentences long,
    # in the third paragraph give me 4 simple statements in numbered list form based on of this text. Text ====== {text}""",
    template= """
        Review this text and summarize it -----{text}
    """,
    input_variables=["text"],
)

In [8]:
chain = prompt | model | output_parser

with open(video.title + '/' + 'AI_' + video.title + '.txt', 'w') as f:
    f.write(chain.invoke(content))
    f.close()


In [9]:
from openai import OpenAI

client = OpenAI()
response = client.audio.speech.create(
    model = 'tts-1',
    voice = 'echo',
       input = open(video.title + '/' + 'AI_' + video.title + '.txt', 'r').read()
    )
response.stream_to_file(video.title + '/' + 'AI_' + video.title + '.mp4')

/tmp/ipykernel_38192/13093331.py:9: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(video.title + '/' + 'AI_' + video.title + '.mp4')
